In [ ]:
!pip install super-gradients==3.1.0
!pip install opencv-python pycocotools matplotlib onnxruntime onnx
!pip install cvzone

In [ ]:
import csv
import cv2
import itertools
import numpy as np
import pandas as pd
import os
import sys
import tempfile
import tqdm
import requests
import torch
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

from super_gradients.training import models

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
import cvzone
import math
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

sc = StandardScaler()

[2023-06-20 09:11:11] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


The console stream is logged into /root/sg_logs/console.log


/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
[2023-06-20 09:11:22] WARNING - __init__.py - Failed to import pytorch_quantization
[2023-06-20 09:11:22] WARNING - calibrator.py - Failed to import pytorch_quantization
[2023-06-20 09:11:22] WARNING - export.py - Failed to import pytorch_quantization
[2023-06-20 09:11:22] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization


In [ ]:
person_model = models.get('yolo_nas_m',
                        num_classes = 1 ,
                        checkpoint_path="/content/drive/MyDrive/person_detection/average_model.pth")

[2023-06-20 09:11:59] INFO - checkpoint_utils.py - Successfully loaded model weights from /content/drive/MyDrive/person_detection/average_model.pth EMA checkpoint.


In [ ]:
# Download model from TF Hub and check out inference code from GitHub
!wget -q -O movenet_thunder.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/float16/4?lite-format=tflite
!git clone https://github.com/tensorflow/examples.git
pose_sample_rpi_path = os.path.join(os.getcwd(), 'examples/lite/examples/pose_estimation/raspberry_pi')
sys.path.append(pose_sample_rpi_path)

# Load MoveNet Thunder model
import utils
from data import BodyPart
from ml import Movenet
movenet = Movenet('movenet_thunder')

# Define function to run pose estimation using MoveNet Thunder.
# You'll apply MoveNet's cropping algorithm and run inference multiple times on
# the input image to improve pose estimation accuracy.
def detect(input_tensor, inference_count=3):
  """Runs detection on an input image.

  Args:
    input_tensor: A [height, width, 3] Tensor of type tf.float32.
      Note that height and width can be anything since the image will be
      immediately resized according to the needs of the model within this
      function.
    inference_count: Number of times the model should run repeatly on the
      same input image to improve detection accuracy.

  Returns:
    A Person entity detected by the MoveNet.SinglePose.
  """
  image_height, image_width, channel = input_tensor.shape

  # Detect pose using the full input image
  movenet.detect(input_tensor.numpy(), reset_crop_region=True)

  # Repeatedly using previous detection result to identify the region of
  # interest and only croping that region to improve detection accuracy
  for _ in range(inference_count - 1):
    person = movenet.detect(input_tensor.numpy(),
                            reset_crop_region=False)

  return person

fatal: destination path 'examples' already exists and is not an empty directory.


In [ ]:
def draw_prediction_on_image(
    image, person, crop_region=None, close_figure=True,
    keep_input_size=False):
  """Draws the keypoint predictions on image.

  Args:
    image: An numpy array with shape [height, width, channel] representing the
      pixel values of the input image.
    person: A person entity returned from the MoveNet.SinglePose model.
    close_figure: Whether to close the plt figure after the function returns.
    keep_input_size: Whether to keep the size of the input image.

  Returns:
    An numpy array with shape [out_height, out_width, channel] representing the
    image overlaid with keypoint predictions.
  """
  # Draw the detection result on top of the image.
  image_np = utils.visualize(image, [person])

  # Plot the image with detection results.
  height, width, channel = image.shape
  aspect_ratio = float(width) / height
  fig, ax = plt.subplots(figsize=(12 * aspect_ratio, 12))
  im = ax.imshow(image_np)

  if close_figure:
    plt.close(fig)

  if not keep_input_size:
    image_np = utils.keep_aspect_ratio_resizer(image_np, (512, 512))

  return image_np

In [ ]:
def get_center_point(landmarks, left_bodypart, right_bodypart):
  """Calculates the center point of the two given landmarks."""

  left = tf.gather(landmarks, left_bodypart.value, axis=1)
  right = tf.gather(landmarks, right_bodypart.value, axis=1)
  center = left * 0.5 + right * 0.5
  return center


def get_pose_size(landmarks, torso_size_multiplier=2.5):
  """Calculates pose size.

  It is the maximum of two values:
    * Torso size multiplied by `torso_size_multiplier`
    * Maximum distance from pose center to any pose landmark
  """
  # Hips center
  hips_center = get_center_point(landmarks, BodyPart.LEFT_HIP,
                                 BodyPart.RIGHT_HIP)

  # Shoulders center
  shoulders_center = get_center_point(landmarks, BodyPart.LEFT_SHOULDER,
                                      BodyPart.RIGHT_SHOULDER)

  # Torso size as the minimum body size
  torso_size = tf.linalg.norm(shoulders_center - hips_center)

  # Pose center
  pose_center_new = get_center_point(landmarks, BodyPart.LEFT_HIP,
                                     BodyPart.RIGHT_HIP)
  pose_center_new = tf.expand_dims(pose_center_new, axis=1)
  # Broadcast the pose center to the same size as the landmark vector to
  # perform substraction
  pose_center_new = tf.broadcast_to(pose_center_new,
                                    [tf.size(landmarks) // (17*2), 17, 2])

  # Dist to pose center
  d = tf.gather(landmarks - pose_center_new, 0, axis=0,
                name="dist_to_pose_center")
  # Max dist to pose center
  max_dist = tf.reduce_max(tf.linalg.norm(d, axis=0))

  # Normalize scale
  pose_size = tf.maximum(torso_size * torso_size_multiplier, max_dist)

  return pose_size


def normalize_pose_landmarks(landmarks):
  """Normalizes the landmarks translation by moving the pose center to (0,0) and
  scaling it to a constant pose size.
  """
  # Move landmarks so that the pose center becomes (0,0)
  pose_center = get_center_point(landmarks, BodyPart.LEFT_HIP,
                                 BodyPart.RIGHT_HIP)
  pose_center = tf.expand_dims(pose_center, axis=1)
  # Broadcast the pose center to the same size as the landmark vector to perform
  # substraction
  pose_center = tf.broadcast_to(pose_center,
                                [tf.size(landmarks) // (17*2), 17, 2])
  landmarks = landmarks - pose_center

  # Scale the landmarks to a constant pose size
  pose_size = get_pose_size(landmarks)
  landmarks /= pose_size

  return landmarks


def landmarks_to_embedding(landmarks_and_scores):
  """Converts the input landmarks into a pose embedding."""
  # Reshape the flat input into a matrix with shape=(17, 3)
  reshaped_inputs = keras.layers.Reshape((17, 3))(landmarks_and_scores)

  # Normalize landmarks 2D
  landmarks = normalize_pose_landmarks(reshaped_inputs[:, :, :2])

  # Flatten the normalized landmark coordinates into a vector
  embedding = keras.layers.Flatten()(landmarks)

  return embedding

In [ ]:
class_names = ['running','sitting','sleeping','standing/walking','working']

In [ ]:
from keras.models import load_model
inputs = tf.keras.Input(shape=(51))
embedding = landmarks_to_embedding(inputs)
l2_strength = 0.001
layer = tf.keras.layers.Dense(512, activation=tf.nn.relu6,kernel_initializer='he_normal')(embedding)
layer = tf.keras.layers.Dropout(0.5)(layer)
layer = tf.keras.layers.Dense(256, activation=tf.nn.relu6,kernel_initializer='he_normal')(layer)
# layer = tf.keras.layers.BatchNormalization()(layer)
layer = tf.keras.layers.Dropout(0.5)(layer)
layer = tf.keras.layers.Dense(128, activation=tf.nn.relu6,kernel_initializer='he_normal')(layer)
# layer = tf.keras.layers.BatchNormalization()(layer)
layer = tf.keras.layers.Dropout(0.5)(layer)
layer = tf.keras.layers.Dense(64, activation=tf.nn.relu6,kernel_initializer='glorot_normal')(layer)
# layer = tf.keras.layers.BatchNormalization()(layer)
layer = tf.keras.layers.Dropout(0.5)(layer)
layer = tf.keras.layers.Dense(32, activation=tf.nn.relu6,kernel_initializer='glorot_normal')(layer)
# layer = tf.keras.layers.BatchNormalization()(layer)
layer = tf.keras.layers.Dense(16, activation=tf.nn.relu6,kernel_initializer='glorot_normal')(layer)
# layer = tf.keras.layers.BatchNormalization()(layer)
outputs = keras.layers.Dense(len(class_names), activation="softmax",kernel_regularizer=tf.keras.regularizers.l2(l2_strength),kernel_initializer='glorot_normal')(layer)
model = keras.Model(inputs, outputs)
model.load_weights('/content/6.hdf5')

In [ ]:
from google.colab.patches import cv2_imshow

video_path_1 = "/content/1.mp4"
video_path_2 = "/content/2.mp4"
# video_path_3 = "/content/fighting.mp4"

result_path_1 = '/content/1_fps30.mp4'
result_path_2 = '/content/2_fps30.mp4'
# result_path_3 = '/content/3_fps30.mp4'

video_list = [video_path_1,video_path_2]
result_list = [result_path_1,result_path_2]

for video_path , result_path in zip(video_list,result_list):
  cap = cv2.VideoCapture(video_path)  # For Video
  frame_width = int(cap.get(3))
  frame_height = int(cap.get(4))
  size = (frame_width, frame_height)
  ans = cv2.VideoWriter(result_path,
                          cv2.VideoWriter_fourcc(*'mp4v'),30, size)

  while True:
    success, img = cap.read()
    if not success :
      break
    l = list(person_model.predict(img))
    results = l[0].prediction.bboxes_xyxy
    conf = l[0].prediction.confidence
    print(results.shape,conf.shape)
    for r,con in zip(results,conf):
      x1 , y1 , x2 , y2 = r
      con= math.ceil((con * 100)) / 100
      a = x1 - frame_width//15
      b = y1 - frame_height//15
      c = x2 + frame_width//15
      d = y2 + frame_height//15
      a,b,c,d = int(a), int(b), int(c), int(d)
      x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
      if a < 0 : a = 0
      if b < 0 : b = 0
      if c >= frame_width : c = frame_width-1
      if d >= frame_height : d = frame_height-1
      imgs = img[a:c,b:d]
      # imgs = img[x1:x2 , y1:y2]
      if imgs.shape[0]!=0 and imgs.shape[1]!=0 :
        imgs = cv2.cvtColor(imgs, cv2.COLOR_BGR2RGB)
        input_tensor = tf.convert_to_tensor(imgs, dtype=tf.float32)
        if input_tensor.shape[0] != 0 and input_tensor.shape[1] != 0 :
          person = detect(input_tensor, inference_count=3)
          pose_landmarks = np.array(
          [[keypoint.coordinate.x, keypoint.coordinate.y, keypoint.score]
          for keypoint in person.keypoints],
          dtype=np.float32)
          coordinates = pose_landmarks.flatten().astype(np.float32).tolist()
          coordinates = np.array(coordinates)
          coordinates = coordinates.reshape(1,-1)
          tensor = tf.convert_to_tensor(coordinates)
          predicted_class_index = np.argmax(model.predict(tensor))
          predicted_class = class_names[predicted_class_index]
          cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 1)
          cvzone.putTextRect(img, f'{predicted_class} {con}', (max(0, x1), max(35, y1)), scale=2, thickness=2)
    ans.write(img)
    cv2.waitKey(1)

  ans.release()
  cap.release()
  cv2.destroyAllWindows()

